In [117]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
%matplotlib inline

In [118]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [119]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SECRET')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

# Plaid Access Token

### 1. Create a client to connect to plaid

In [120]:
INSITUTION_ID = "ins_109508"

In [121]:
# YOUR CODE HERE
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SECRET, public_key=PLAID_PUBLIC_KEY, environment='sandbox')


### 2. Generate a public token

In [122]:
# Create public token to be exchanged for institution access token
create_tkn_response = client.Sandbox.public_token.create(INSITUTION_ID, ['transactions','income','assets'])

### 3. Exchange the public token for an access token

In [123]:
# Exchange public token for access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

# Store access token as variable
access_token = exchange_response['access_token']

---

# Account Transactions with Plaid

### 1.Transactions for the last 90 days

In [130]:
# Transactions for institution for specific date range
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

# Transactions for date range
transaction_response = client.Transactions.get(access_token,start_date,end_date)
transactions90 = transaction_response['transactions']

### 2. Categories for each transaction

In [126]:
categories = [transactions90[i]['category']
     for i in range(len(transactions90))]
print(categories)

[['Travel', 'Airlines and Aviation Services'], ['Travel', 'Car Service', 'Ride Share'], ['Food and Drink', 'Restaurants'], ['Payment'], ['Food and Drink', 'Restaurants'], ['Shops', 'Sporting Goods'], ['Payment', 'Credit Card'], ['Travel', 'Car Service', 'Ride Share'], ['Transfer', 'Debit'], ['Transfer', 'Deposit'], ['Recreation', 'Gyms and Fitness Centers'], ['Travel', 'Airlines and Aviation Services'], ['Food and Drink', 'Restaurants'], ['Food and Drink', 'Restaurants', 'Coffee Shop'], ['Food and Drink', 'Restaurants'], ['Transfer', 'Credit'], ['Travel', 'Airlines and Aviation Services'], ['Travel', 'Car Service', 'Ride Share'], ['Food and Drink', 'Restaurants'], ['Payment'], ['Food and Drink', 'Restaurants'], ['Shops', 'Sporting Goods'], ['Payment', 'Credit Card'], ['Travel', 'Car Service', 'Ride Share'], ['Transfer', 'Debit'], ['Transfer', 'Deposit'], ['Recreation', 'Gyms and Fitness Centers'], ['Travel', 'Airlines and Aviation Services'], ['Food and Drink', 'Restaurants'], ['Food a

### 3. New DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

In [128]:
dates = [transactions90[i]['date']
     for i in range(len(transactions90))]

names = [transactions90[i]['name']
     for i in range(len(transactions90))]

amounts = [transactions90[i]['amount']
     for i in range(len(transactions90))]

category1 = []
for x in categories:
    category1.append(x[0])

df = pd.DataFrame(list(zip(dates, names, amounts, category1)), 
               columns =['Dates', 'Name', 'Amount', 'Category']) 
df 

,Dates,Name,Amount,Category
0,2019-09-03,United Airlines,500.00,Travel
1,2019-09-01,Uber 072515 SF**POOL**,6.33,Travel
2,2019-08-29,Tectra Inc,500.00,Food and Drink
3,2019-08-28,AUTOMATIC PAYMENT - THANK,2078.50,Payment
4,2019-08-28,KFC,500.00,Food and Drink
5,2019-08-28,Madison Bicycle Shop,500.00,Shops
6,2019-08-19,CREDIT CARD 3333 PAYMENT *//,25.00,Payment
7,2019-08-19,Uber 063015 SF**POOL**,5.40,Travel
8,2019-08-18,ACH Electronic CreditGUSTO PAY 123456,5850.00,Transfer
9,2019-08-18,CD DEPOSIT .INITIAL.,1000.00,Transfer


### 4. Converting the data types to the appropriate types 

In [131]:
# YOUR CODE HERE
df['datetime'] = pd.to_datetime(df['Dates'])
df_indexed = df.set_index('datetime')
df_indexed.drop(['Dates'], axis=1, inplace=True)
df_indexed.head()

,Name,Amount,Category
datetime,,,
2019-09-03,United Airlines,500.00,Travel
2019-09-01,Uber 072515 SF**POOL**,6.33,Travel
2019-08-29,Tectra Inc,500.00,Food and Drink
2019-08-28,AUTOMATIC PAYMENT - THANK,2078.50,Payment
2019-08-28,KFC,500.00,Food and Drink


In [132]:
df_indexed.Amount.dtype

dtype('float64')

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [ ]:
# YOUR CODE HERE

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calulate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# YOUR CODE HERE

### Calculate the expenses per month

In [ ]:
# YOUR CODE HERE